In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/university-students-complaints-and-reports/Datasetprojpowerbi.csv


In [2]:
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
from nltk.tokenize import word_tokenize
nltk.download("punkt")
from nltk.stem import PorterStemmer
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

df = pd.read_csv("/kaggle/input/university-students-complaints-and-reports/Datasetprojpowerbi.csv")

df["Reports_lower"] = df["Reports"].str.lower()

translator = str.maketrans(" "," ",string.punctuation)
df["Reports_nopunct"] = df["Reports_lower"].str.translate(translator)

english_stopwords = set(stopwords.words("english"))
custom_stopwords = {"im","really","student","like","feel","ive"}
combined_stopwords = english_stopwords.union(custom_stopwords)
def remove_stopwords(text):
    words = word_tokenize(text.lower())
    improved_text = [word for word in words if word not in combined_stopwords and word not in translator and not word.isdigit()]
    return " ".join(improved_text)

df["cleaned_Reports"] = df["Reports_nopunct"].apply(remove_stopwords)

stemmer = PorterStemmer()
def stem_text(text):
    words = word_tokenize(text)
    stemmed_words = [stemmer.stem(word) for word in words]
    return " ".join(stemmed_words)

df["stemmed_Reports"] = df["cleaned_Reports"].apply(stem_text)

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df["stemmed_Reports"])
tfidf_df = pd.DataFrame(X.toarray(), columns = vectorizer.get_feature_names_out())

print(tfidf_df.head(10).tail(10))

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
   15minut  30minut  abil       abl  abroad    academ  accent  accept  \
0      0.0      0.0   0.0  0.263485     0.0  0.194256     0.0     0.0   
1      0.0      0.0   0.0  0.000000     0.0  0.000000     0.0     0.0   
2      0.0      0.0   0.0  0.000000     0.0  0.000000     0.0     0.0   
3      0.0      0.0   0.0  0.000000     0.0  0.169476     0.0     0.0   
4      0.0      0.0   0.0  0.000000     0.0  0.000000     0.0     0.0   
5      0.0      0.0   0.0  0.000000     0.0  0.000000     0.0     0.0   
6      0.0      0.0   0.0  0.000000     0.0  0.000000     0.0     0.0   
7      0.0      0.0   0.0  0.000000     0.0  0.000000     0.0     0.0   
8      0.0      0.0   0.0  0.000000     0.0  0.000000     0.0     0.0   
9      0.0      0.0   0.0  0.0

In [3]:
sum_tfidf = np.array(tfidf_df.sum(axis=0)).flatten()
words_tfidf_df = pd.DataFrame(list(zip(tfidf_df,sum_tfidf)), columns = ["word", "tfidf_score"])
most_frequent_words = words_tfidf_df.sort_values(by="tfidf_score", ascending=False)

print(most_frequent_words.head(20))

           word  tfidf_score
1103    student    36.850763
1228    univers    28.569606
8        access    28.494155
5        academ    28.289367
655       limit    27.049631
439        find    26.602705
786      option    26.039072
83        avail    25.734455
688        make    25.623847
744        need    25.107747
784    opportun    23.979267
179       class    23.335904
1166       time    22.860928
241       cours    22.815588
615         job    22.703614
293   difficult    22.190598
514        hard    22.053396
781       onlin    21.959751
145       campu    21.881768
1285       wish    20.266363
